# 第5章 决策树
顾名思义,"决策树"就是按照建立分支来进行决策的方法,即根据啥(往往是根据某个属性的某个值作为指标，比如“年龄大于20岁的”或“身高大于1米5的”)来区分不同类别的数据,也是一种监督式机器学习算法,从编程角度来看,可以看作是一系列的"if... then... else..."的集合,这个模型非常直观,也即可读性非常好,决策树的学习通常包含三个步骤:**特征选择**,**决策树生成**,**决策树的剪枝**.  
给定数据集
>$$D = \{ (x_1,y_1),(x_2,y_2),...,(x_N,y_N)\}$$
其中,$x_i=(x_i^{(1)},x_i^{(2)},...,x_i^{(n)})^T$为输入实例(特征向量),$n$为特征的个数,$y_i \in \{1,2,...,K\}$为类别标记,$i = 1,2,...,N$,$N$为样本容量

目标
>构建决策树,使之能改对大量的实例进行正确分类（而为什么要引入决策树的剪枝呢？是要让这颗树尽可能地好，那什么叫好呢？就是用尽可能少的分叉来高效地区分更多的数据）  
(本质是构建一组规则,这组规则可以对每个特征进行划分,但是又不能划分太好,不然容易过拟合,泛化能力就会变差)

## 5.1数据集

首先建立课本上的数据集合本身  
>年龄定义"青年|中年|老年"分别为"1|2|3",  
是否有工作,定义"是|否"为"1|2",  
有自己的房子,定义"是|否"为"1|2",  
信贷情况"一般|好|非常好"为"1|2|3",  
类别"是|否"为"1|2"

In [7]:
import numpy as np
x = np.asarray([[1,2,2,1],[1,2,2,2],[1,1,2,2],[1,1,1,1],[1,2,2,1],
               [2,2,2,1],[2,2,2,2],[2,1,1,2],[2,2,1,3],[2,2,1,3],
               [3,2,1,3],[3,2,1,2],[3,1,2,2],[3,1,2,3],[3,2,2,1]])
y = np.asarray([2,2,1,1,2,2,2,1,1,1,1,1,1,1,2])

> 此时某些用心的读者也许还会发现,给了这些数据之后,实际可以尝试性地按照x的某一列属性作为指标,给定阈值,阈值之上的为某个列别标签,阈值之下的为另一个类别标签,但是问题来啦:选择哪个属性最好呢?
这里不得不说到"信息增益"的问题了,这个东西可以指导模型选择最合适的属性对原始数据进行第一阶的划分.

>这里需要对“信息增益”进行一些讲解，信息增益可以这么理解：如果对一件事进行描述，并且需要10次才能描述完整，那么实际每次描述的时候，描述所涵盖的信息量是不相同的，信息增益，就可以理解为一次描述过程的量。对于这里的决策树来说，就是每次if和else所能带有的信息。实际的定义和形式化描述如下：

---
## 5.2信息熵和信息增益
### 5.2.1信息熵，经验熵，相对熵
（1）＂**熵**＂的定义：$$H(X) = -\sum_{i=1}^nP_i\log{P_i}$$
>观察上式可以得知，熵只和数据分布有关，所以，$H(X)$可以直接写作$H(p)$，且从定义可以验证
$$0 \leq H(p) \leq \log n$$
>（关于＂熵＂为什么要这么定义，这里给一下思路提示：比如，如果要定义一个三位数，实际只用３个字符存储空间就可以了，这也就是为什么取log的原因了，实际表示的是描述某个对象所需要的基本元素的个数，更详细的关于信息论的定义可以自行学习香浓的那一套信息理论，这里我也不懂很多，只给点提示，另外，这里结合课本给出求解信息熵的代码）

In [8]:
from collections import Counter #计数插件
def cal_entropy(x,i):
    """
    计算熵值
    x：数据集
    ｉ：第ｉ个特征
    """
    if x.ndim == 1:
        x = x.reshape((len(x),1)) # 如果是一维的，那么定义为(len(x)行1列)的数据分布
    col_i_data = x[:,i] # 第ｉ列数据
    uniquevalue = np.unique(col_i_data) # 该属性下查找所有可能的值
    H_X = 0 # 统计熵的加和
    for j in range(len(uniquevalue)): # 
        i_p = Counter(col_i_data)[j+1]/len(col_i_data) # 对每个属性值计数并计算概率值（占比）
        if i_p != 0:
            i_logp = np.log2(i_p) # 对数值
            H_X = H_X + (-i_p*i_logp) # 累加
        elif i_p == 0:
            H_X = H_X + 1 # 对于概率值为0的，定义0*log0=1，并累加
    return H_X

（2）**经验熵**的定义：
>实际是对$y$求熵，即$H(Y)$

In [9]:
print('经验熵 =',cal_entropy(y,0))

经验熵 = 0.970950594455


（3）**"相对熵"**的定义：
>$$H(Y|X) = \sum_{i=1}^n H(Y|X=x_i)$$
表示在已知随机变量$X$的条件下随机变量$Y$的不确定性．

In [10]:
def cal_mutualinformation(x,y,i):
    """
    计算第ｉ列的条件熵
        x: 数据
        y: 类别标签
        i: 第ｉ列
    """
    col_i_data = x[:,i]
    data = np.column_stack((col_i_data,y)) # 合并了一个数组，[ｘ_i,y]
    uniquevalue = np.unique(col_i_data) # 查找所有可能的值
    H_YconX = 0 # 统计熵的加和
    H_YconX = 0
    for j in range(len(uniquevalue)):
        D_i = data[col_i_data == uniquevalue[j],:]
        uniqueclass = np.unique(y)
        factor2 = 0
        for k in range(len(uniqueclass)):# H(Di) = sum(|Dik|/Di * log |Dik|/Di)
            D_ik = D_i[D_i[:,1] == uniqueclass[k],:]
            if (D_ik.shape[0] == 0):
                temp2 = 0
            else:
                temp2 = - D_ik.shape[0]/D_i.shape[0] * np.log2(D_ik.shape[0]/D_i.shape[0])
                # print(temp2)
            factor2 = factor2 + temp2
        H_YconX = H_YconX + len(D_i)/len(col_i_data) * factor2
    return H_YconX

### 5.2.2信息增益

信息增益为
>$$g(D,A) = H(D)-H(D|A)$$
表示$A$特征对训练数据$D$的信息增益

In [12]:
for m in range(x.shape[1]):
    print('第',m,'个特征的信息增益值为：',cal_entropy(y,0) - cal_mutualinformation(x,y,m))

第 0 个特征的信息增益值为： 0.0830074998558
第 1 个特征的信息增益值为： 0.323650198152
第 2 个特征的信息增益值为： 0.419973094022
第 3 个特征的信息增益值为： 0.362989562537


>将上面的过程定义成函数

In [19]:
def cal_information_gain(x,y):
    """
    计算每个特征下的信息增益值
    """
    result = []
    for i in range(x.shape[1]):
        value = cal_entropy(y,0) - cal_mutualinformation(x,y,i)
        result.append(value)
    return result
cal_information_gain(x,y)

[0.083007499855768829,
 0.32365019815155627,
 0.41997309402197491,
 0.36298956253708536]

>所以上面选择增益最大的(0.419973)，即第２个特征（有无房子）

---
# 5.3 定义自己的决策树函数

In [ ]:
class myDecisionTree():
    def train():
        """
        训练
        """    
    
    def predict():
        """
        测试
        """    
    
    def cal_entropy():
        """
        计算熵
        """    
    
    def cal_mutualinformation():
        """
        计算互信息值
        """
    
    def cal_information_gain():
        """
        """